# Анализ отзывов клиентов
На примере сайта [trustpilot.com](https://www.trustpilot.com/review/flixbus.com) и компании flixbus

Идея работника фирмы flixbus **Shukhrat Khodjaev**
https://clck.ru/V8LDY



In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from time import sleep
import datetime
import json

def clean_string(column):
    return column.apply(lambda x: x.replace("\n",'',2)).apply(lambda x: x.replace('  ',''))
    
# процедура парсит отзывы с сайта https://www.trustpilot.com
# работала 28 мая 2021 года

url_main='https://www.trustpilot.com'

def parse_reviews(company_url, page_max=2, sleep_time = 0.5):
# адрес основного сайта и начальный запрос
  url0=url_main; path='/review/' + company_url

# списки вытаскиваемых данных
  names = []     # имя пользователя
  ratings = []   # рейтинг от пользователя (от 1 до 5)
  headers = []   # заголовок отзыва
  reviews = []   # текст отзыва
  dates = []     # дата отзыва
  countries = [] # страна пользователя

  page=0; rec=0;
  while(path):
    page += 1
    if (page_max > 0) and (page > page_max): # отрабатываем нужное количество страниц, если задано
      break
    url = str(url0) + str(path)              # формируем полный адрес
    http = requests.get(f'{url}')            # вытаскиваем страницу

    bsoup = BeautifulSoup(http.text, 'html.parser') # варим суп (раскладываем на элементы)

    # вытаскиваем ссылку на следующую страницу - переменная path
    href = bsoup.find('a', href=True, rel="next")
    path = href['href'] if (href) else ''

    # вытаскиваем данные по переменным в соотвествующие контейнеры
    # размерности всех контейнеров должны быть одинаковыми и равны количеству отзывов на одной странице
    country_container = bsoup.find_all('div', class_ = 'consumer-information__location')
    nrec=len(country_container)

    print("Парсим страницу", page, ", записи",rec+1,"-",rec+nrec,"( адрес=",url,")")
    rec += nrec;

    review_container = bsoup.find_all('div', class_ = 'review-content__body')
    nrec1=len(review_container)
    if (nrec1 != nrec): 
      print("ОШИБКА: число отзывов=",nrec1,"!=",nrec,"записей по стране!")
    rating_container = bsoup.find_all('div',class_ = "star-rating star-rating--medium")
    nrec1=len(rating_container)
    if (nrec1 != nrec): 
      print("ОШИБКА: число рейтингов=",nrec1,"!=",nrec,"записей по стране!")
    date_container = bsoup.find_all('div',class_ = "review-content-header__dates")
    nrec1=len(date_container)
    if (nrec1 != nrec): 
      print("ОШИБКА: число дат=",nrec1,"!=",nrec,"записей по стране!")
    name_container = bsoup.find_all('div',class_ = "consumer-information__name")
    nrec1=len(name_container)
    if (nrec1 != nrec): 
      print("ОШИБКА: число пользователей=",nrec1,"!=",nrec,"записей по стране!")

    # извлечение и запись переменных одного отзыва в общие списки
    for x in range(nrec):
      countries.append(country_container[x].text)
      names.append(name_container[x].text)
      headers.append(review_container[x].h2.a.text)
      try:
        reviews.append(review_container[x].p.text)
      except AttributeError:
        reviews.append("")
      dc = json.loads(date_container[x].script.text)
      dates.append(datetime.datetime.strptime(dc["publishedDate"][0:10],'%Y-%m-%d').date())
      ratings.append(rating_container[x].img['alt'].split(' ')[0])

  # формирование dataframe из собранных списков данных
  rev_df = pd.DataFrame(list(zip(names, headers, reviews, ratings, dates, countries)),
                  columns = ['Name','Header','Review','Rating', 'Date', 'Country'])
    
  # очистка от мусорных знаков и приведение к нужному типу переменных
  rev_df.Name = clean_string(rev_df.Name)
  rev_df.Header = clean_string(rev_df.Header)
  rev_df.Review = clean_string(rev_df.Review)
  rev_df.Rating = rev_df.Rating.astype('int')
  rev_df.Date = pd.to_datetime(rev_df.Date)
  rev_df.Country = clean_string(rev_df.Country)
    
  return rev_df # конец parse_reviews
    
print("Парсер parse_reviews для сайта",url_main,"загружен!")

In [ ]:
df = parse_reviews("www.flixbus.com",5)
# df = parse_reviews("www.nationalexpress.com",20)
df

## Сохранение данных 

Для последующей обработки, чтобы не мучить сайт, данные можно сохранить

In [80]:
df.to_csv("trustpilot_flixbus_reviews.csv")

## Гистограмма рейтинга
Видно, что больше всего отзывов оставляют самые обиженные и самые довольные клиенты.

In [ ]:
import matplotlib.pyplot as plt
rt = df["Rating"]
rt.plot(kind="hist")

## Отзывы по странам

In [ ]:
import seaborn as sns
df1 = df.groupby("Country").count().reset_index()
df1.head()
top_20 = df1.sort_values(by=['Review'], ascending=False).head(20)
plt.figure(figsize=(12,10))
plot = sns.barplot(top_20['Review'], top_20['Country'])
for i,(value,name) in enumerate(zip(top_20['Review'],top_20['Country'])):
  plot.text(value,i-0.05,f'{value:,.0f}',size=10)
plt.show()


## Средний отзыв по странам

In [ ]:
df2a = df.groupby("Country").agg({
    'Review' : lambda x: x.count(), 
    'Rating' : lambda x: x.mean(),
    'Country': lambda x: x.max()})
#df2a.rename(columns={'Review': 'NRev',      # число отзывов 
#                         'Rating': 'AveRT' # средний рейтинг
#                    }, inplace=True)
df2a.head()

df2 = df2a[df2a["Review"]>20] # смотрим только страны с заметным числом отзывов

df2.head()

rt = df2.sort_values(by=['Rating'], ascending=False)
plt.figure(figsize=(12,10))
plot = sns.barplot(rt['Rating'],rt['Country'])
for i,(value,name) in enumerate(zip(rt['Rating'],rt['Country'])):
  plot.text(value,i-0.05,f'{value:,.2f}',size=10)
plt.show()

In [ ]:
df['my'] = df['Date'].dt.to_period('M') # добавление месячного интервала
df.head()

df3a = df.groupby("my").agg({
    'Review' : lambda x: x.count()/100, 
    'Rating' : lambda x: x.mean(),
    'my': lambda x: x.max()})

df3a.head()

plt.figure(figsize=(12,50))
df3a.plot(x="my", y=["Review", "Rating"])
plt.legend(["Средний рейтинг", "Число отзывов/100"]);